In [1]:
import requests
import re
from bs4 import BeautifulSoup
from random import randint

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException


import time
import pandas as pd
from tqdm import tqdm_notebook
from tqdm import trange

In [2]:
# 평점 순 전체 기업의 링크를 리스트로 가져온다.(bs4)

com_link_ls = []

for nx_pg in trange(1, 472):
    com_url = 'https://www.jobplanet.co.kr/companies?sort_by=review_avg_cache&page={}'.format(nx_pg)
    html = requests.get(com_url)
    soup = BeautifulSoup(html.text, 'lxml')
    com_ls = soup.select('dl dt a')

    for ls in com_ls:
        com_link_ls.append(ls.attrs['href'])

 37%|██████████████▉                         | 176/471 [05:14<08:53,  1.81s/it]

KeyboardInterrupt: 

In [11]:
com_link_ls = com_link_ls[:1203]

In [12]:
jp_link = 'https://www.jobplanet.co.kr'

for i in range(len(com_link_ls)):
    com_link_ls[i] = jp_link + com_link_ls[i]
    com_link_ls[i] = com_link_ls[i].replace('info', 'landing')

In [13]:
com_link_ls

['https://www.jobplanet.co.kr/companies/90364/landing/%ED%8E%98%EC%9D%B4%EC%8A%A4%EB%B6%81%EC%BD%94%EB%A6%AC%EC%95%84?',
 'https://www.jobplanet.co.kr/companies/112614/landing/%EC%95%A0%ED%84%B0%EB%AF%B8?',
 'https://www.jobplanet.co.kr/companies/313882/landing/%EB%B2%84%EC%A6%88%EB%B9%8C?',
 'https://www.jobplanet.co.kr/companies/308098/landing/%EA%B8%88%EC%9C%B5%EB%B3%B4%EC%95%88%EC%9B%90?',
 'https://www.jobplanet.co.kr/companies/89723/landing/%EB%9D%BC%EC%9D%B4%EC%97%87%EA%B2%8C%EC%9E%84%EC%A6%88%EC%BD%94%EB%A6%AC%EC%95%84?',
 'https://www.jobplanet.co.kr/companies/80176/landing/%EB%93%9C%EB%9D%BC%EB%A7%88%EC%95%A4%EC%BB%B4%ED%8D%BC%EB%8B%88?',
 'https://www.jobplanet.co.kr/companies/53479/landing/%EC%BC%80%EC%9D%B4%EC%97%90%EC%8A%A4%EC%97%90%EC%8A%A4%ED%95%B4%EC%9A%B4?',
 'https://www.jobplanet.co.kr/companies/94877/landing/%EC%9C%A0%EB%8B%88%ED%8B%B0%ED%85%8C%ED%81%AC%EB%86%80%EB%A1%9C%EC%A7%80%EC%8A%A4%EC%BD%94%EB%A6%AC%EC%95%84?',
 'https://www.jobplanet.co.kr/companies/3280/la

In [15]:
a = pd.DataFrame(com_link_ls, index=None)

In [16]:
a.to_csv('기업소개링크.csv')

In [415]:
len(com_link_ls[1450:])

3905

In [65]:
a = pd.DataFrame(com_link_ls[122:1300])

In [67]:
a.to_csv('122_1300.csv', index=False)

In [3]:
b = pd.read_csv('122_1300.csv')
c = list(b['0'])

In [26]:
driver = webdriver.Chrome('chromedriver.exe')

In [27]:
driver.get('https://www.jobplanet.co.kr/companies?sort_by=review_avg_cache')

In [28]:
# button = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """/html/body/div[5]/div[1]/header/div/div[2]/a[1]/span""")))
# button.click()

# 로그인 창으로 이동
button = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """/html/body/div[5]/div[1]/header/div/div[2]/a[1]/span""")))
button.click()


# 로그인 정보 입력
input_id = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="user_email"]""")))
input_pswd = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="user_password"]""")))
input_id.send_keys('201302099@inu.ac.kr')
input_pswd.send_keys('2013020999206')

# 로그인 버튼 클릭
login = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="signInSignInCon"]/div[2]/div/section[2]/fieldset/button""")))
login.click()

In [29]:
data = []
error_ls = []

for index, start_url in enumerate(c[237:]):
        
    driver.get(start_url)
    
    # 링크 순서
    print('평점 순 링크 순서 : ', index)

    # 기업명
    name = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """/html/body/div[5]/div[2]/div[1]/div/div[2]/div/div/div[1]/div[2]/div[1]/h1/a"""))).text
    print('기업 이름 : ', name)
    
    # 산업분류
    category = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """/html/body/div[5]/div[2]/div[1]/div/div[2]/div/div/div[1]/div[2]/div[2]/div[2]/span[1]"""))).text
#     print('산업 분류 : ', category)
    
    # 직원의 기업 추천율
    recommend_rate = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="mainContents"]/div[1]/div[1]/div/div[3]/div[1]/div[1]/span"""))).text
#     print('직원의 기업 추천율 : ', recommend_rate)
    
    # 직원이 전망하는 성장 가능성
    grow_rate = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="mainContents"]/div[1]/div[1]/div/div[3]/div[2]/div[1]/span"""))).text
#     print('직원이 전망하는 성장 가능성 : ', grow_rate)
    
    # 이 기업의 CEO 지지율
    CEO_rate = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="mainContents"]/div[1]/div[1]/div/div[3]/div[3]/div[1]/span"""))).text
#     print('이 기업의 CEO 지지율 : ', CEO_rate)
    
    # 기업의 총 평점
    point = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """/html/body/div[5]/div[2]/div[1]/div/div[2]/div/div/div[1]/div[2]/div[2]/div[1]/span"""))).text
#     print('기업 총 평점 : ', point)
    
    # 승진 기회 및 가능성
    ch = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="mainContents"]/div[1]/div[1]/div/div[2]/dl/dd[1]/span[2]"""))).text
#     print('승진 기회 및 가능성 : ', ch)
    
    # 복지 및 급여
    sal = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="mainContents"]/div[1]/div[1]/div/div[2]/dl/dd[2]/span[2]"""))).text
#     print('복지 및 급여 : ', sal)
    
    # 업무와 삶의 균형
    bal = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="mainContents"]/div[1]/div[1]/div/div[2]/dl/dd[3]/span[2]"""))).text
#     print('업무와 삶의 균형 : ', bal)
    
    # 사내문화
    corp = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="mainContents"]/div[1]/div[1]/div/div[2]/dl/dd[4]/span[2]"""))).text
#     print('사내문화 : ', corp)
    
    # 경영진
    excutive = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="mainContents"]/div[1]/div[1]/div/div[2]/dl/dd[5]/span[2]"""))).text
#     print('경영진 : ', excutive)
    
        
    # 총 리뷰 갯수
    review_cnt = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="viewCompaniesMenu"]/ul/li[2]/a/span"""))).text    
    cnt = int(review_cnt)
    print('기업 리뷰 갯수 : ', cnt)
    print('=====================================')
    
    
    try:
        # 리뷰의 갯수가 5단위로 나누어 지면 전체 리뷰를 5로 나눈 몪 만큼 페이지 이동
        if cnt % 5 == 0:
            for n_pg in range(0, int(cnt/5)):
                n = randint(8, 10)
                time.sleep(n)


                for i in range(1, 6):
                    #제목, 장점, 단점, 직무, 재직상태, 지역, 개인평점                                                                                         
                    title = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/div[1]/h2""".format(i)).text  
                    adv = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/dl/dd[1]/span""".format(i)).text  
                    dis = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/dl/dd[2]/span""".format(i)).text
                    to_excutive = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/dl/dd[3]/span""".format(i)).text
                    role = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/span[2]""".format(i)).text
                    state = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/span[4]""".format(i)).text
                    loc = driver.find_element_by_xpath(""" //*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/span[6]""".format(i)).text
                    like = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/div[2]/a[1]/span[2]""".format(i)).text 
                    
                    # 각 리뷰의 평점
                    pt_ls = driver.find_elements_by_class_name("star_score")
                    pt = []
                    for score in pt_ls:
                        score = score.get_attribute("style")
                        socre_ls = re.findall("\d+", score)

                        if socre_ls[0] == '100':
                            pt.append('5')

                        elif socre_ls[0] == '80':
                            pt.append('4')

                        elif socre_ls[0] == '60':
                            pt.append('3')

                        elif socre_ls[0] == '40':
                            pt.append('2')

                        else :
                            pt.append('1')


                    pt = [y for x in pt for y in x]
                    pt.insert(0, 'index')
                    
                    
                    if len(loc) <= 3:
                        data.append([index, name, point, category, recommend_rate, grow_rate, CEO_rate, ch, sal, bal, corp, excutive, title, adv ,dis, to_excutive, pt[i], role, state, loc, like])
                    else :
                        data.append([index, name, point, category, recommend_rate, grow_rate, CEO_rate, ch, sal, bal, corp, excutive, title, adv ,dis, to_excutive, pt[i], role, state, '', like])
                button = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """/html/body/div[5]/div[3]/div/div[1]/div[3]/article[2]/div/div/div/div/article/a[7]""")))
                button.click() 

            print('현재 전체 데이터 갯수 : ', len(data))
            print('-------------------------------------')




        # 리뷰의 갯수가 5단위로 나누어 지면 전체 리뷰를 5로 나눈 몫에 더하기 2 만큼 페이지 이동    


        else :
            for n_pg in range(0, int(cnt/5)+2):
                n = randint(8, 10)
                time.sleep(n)
                
                for i in range(1, 6):
                    #제목, 장점, 단점, 직무, 재직상태, 지역, 개인평점
                    title = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/div[1]/h2""".format(i)).text  
                    adv = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/dl/dd[1]/span""".format(i)).text  
                    dis = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/dl/dd[2]/span""".format(i)).text
                    to_excutive = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/dl/dd[3]/span""".format(i)).text
                    role = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/span[2]""".format(i)).text
                    state = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/span[4]""".format(i)).text
                    loc = driver.find_element_by_xpath(""" //*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/span[6]""".format(i)).text
                    like = driver.find_element_by_xpath("""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/div[2]/a[1]/span[2]""".format(i)).text 
                    
                    # 각 리뷰의 평점
                    pt_ls = driver.find_elements_by_class_name("star_score")
                    pt = []
                    for score in pt_ls:
                        score = score.get_attribute("style")
                        socre_ls = re.findall("\d+", score)

                        if socre_ls[0] == '100':
                            pt.append('5')

                        elif socre_ls[0] == '80':
                            pt.append('4')

                        elif socre_ls[0] == '60':
                            pt.append('3')

                        elif socre_ls[0] == '40':
                            pt.append('2')

                        else :
                            pt.append('1')


                    pt = [y for x in pt for y in x]
                    pt.insert(0, 'index')



        #                     title = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/div[1]/h2""".format(i)).text
        #                     adv = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/div/dl/dd[1]/span""".format(i)).text
        #                     dis = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/dl/dd[2]/span""".format(i)).text   
        #                     role = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/span[2]""".format(i)).text
        #                     state = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/span[4]""".format(i)).text
        #                     loc = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """ //*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[1]/span[6]""".format(i)).text
        #                     like = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,"""//*[@id="viewReviewsList"]/div/div/div/section[{}]/div/div[2]/div/div[2]/a[1]/span[2]""".format(i)).text   


                    if len(loc) <= 3:
                        data.append([index, name, point, category, recommend_rate, grow_rate, CEO_rate, ch, sal, bal, corp, excutive, title, adv ,dis, to_excutive, pt[i], role, state, loc, like])
                    else :
                        data.append([index, name, point, category, recommend_rate, grow_rate, CEO_rate, ch, sal, bal, corp, excutive, title, adv ,dis, to_excutive, pt[i], role, state, '', like])

                button = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH, """/html/body/div[5]/div[3]/div/div[1]/div[3]/article[2]/div/div/div/div/article/a[7]""")))
                button.click() 
                
            print('현재 전체 데이터 갯수 : ', len(data))
            print('-------------------------------------')
        
    except Exception as ex:
        print('현재 전체 데이터 갯수 : ', len(data))
        print('-------------------------------------')
        pass


평점 순 링크 순서 :  0
기업 이름 :  포스코에너지(주)
기업 리뷰 갯수 :  117
현재 전체 데이터 갯수 :  117
-------------------------------------
평점 순 링크 순서 :  1
기업 이름 :  한국국제협력단
기업 리뷰 갯수 :  392
현재 전체 데이터 갯수 :  509
-------------------------------------
평점 순 링크 순서 :  2
기업 이름 :  한국베링거인겔하임(주)
기업 리뷰 갯수 :  63
현재 전체 데이터 갯수 :  572
-------------------------------------
평점 순 링크 순서 :  3
기업 이름 :  한성아이티엘(주)
기업 리뷰 갯수 :  25
현재 전체 데이터 갯수 :  597
-------------------------------------
평점 순 링크 순서 :  4
기업 이름 :  (주)이디야
기업 리뷰 갯수 :  506
현재 전체 데이터 갯수 :  1103
-------------------------------------
평점 순 링크 순서 :  5
기업 이름 :  (주)트리플하이엠
기업 리뷰 갯수 :  91
현재 전체 데이터 갯수 :  1194
-------------------------------------
평점 순 링크 순서 :  6
기업 이름 :  경기중소기업종합지원센터
기업 리뷰 갯수 :  28
현재 전체 데이터 갯수 :  1222
-------------------------------------
평점 순 링크 순서 :  7
기업 이름 :  배달의민족
기업 리뷰 갯수 :  205
현재 전체 데이터 갯수 :  1427
-------------------------------------
평점 순 링크 순서 :  8
기업 이름 :  삼천리(주)
기업 리뷰 갯수 :  59
현재 전체 데이터 갯수 :  1486
-------------------------------------
평점 순 링크 순서 :  9
기업 이름 :  

현재 전체 데이터 갯수 :  11074
-------------------------------------
평점 순 링크 순서 :  111
기업 이름 :  두산중공업(주)
기업 리뷰 갯수 :  569
현재 전체 데이터 갯수 :  11643
-------------------------------------
평점 순 링크 순서 :  112
기업 이름 :  로보티즈(주)
기업 리뷰 갯수 :  28
현재 전체 데이터 갯수 :  11671
-------------------------------------
평점 순 링크 순서 :  113
기업 이름 :  부산도시공사
기업 리뷰 갯수 :  23
현재 전체 데이터 갯수 :  11694
-------------------------------------
평점 순 링크 순서 :  114
기업 이름 :  삼성여행사(주)
기업 리뷰 갯수 :  24
현재 전체 데이터 갯수 :  11718
-------------------------------------
평점 순 링크 순서 :  115
기업 이름 :  애드웹커뮤니케이션
기업 리뷰 갯수 :  23
현재 전체 데이터 갯수 :  11724
-------------------------------------
평점 순 링크 순서 :  116
기업 이름 :  에스비아이저축은행(주)
기업 리뷰 갯수 :  97
현재 전체 데이터 갯수 :  11821
-------------------------------------
평점 순 링크 순서 :  117
기업 이름 :  에이아이지손해보험(주)
기업 리뷰 갯수 :  63
현재 전체 데이터 갯수 :  11884
-------------------------------------
평점 순 링크 순서 :  118
기업 이름 :  에이티커니코리아(유)
기업 리뷰 갯수 :  83
현재 전체 데이터 갯수 :  11967
-------------------------------------
평점 순 링크 순서 :  119
기업 이름 :  영림원소프트랩(주)
기업 리

-------------------------------------
평점 순 링크 순서 :  220
기업 이름 :  (주)흥아
기업 리뷰 갯수 :  29
현재 전체 데이터 갯수 :  24697
-------------------------------------
평점 순 링크 순서 :  221
기업 이름 :  대한법률구조공단
기업 리뷰 갯수 :  55
현재 전체 데이터 갯수 :  24752
-------------------------------------
평점 순 링크 순서 :  222
기업 이름 :  메르세데스벤츠코리아(주)
기업 리뷰 갯수 :  97
현재 전체 데이터 갯수 :  24849
-------------------------------------
평점 순 링크 순서 :  223
기업 이름 :  엘에스니꼬동제련(주)
기업 리뷰 갯수 :  69
현재 전체 데이터 갯수 :  24918
-------------------------------------
평점 순 링크 순서 :  224
기업 이름 :  자트코코리아엔지니어링(주)
기업 리뷰 갯수 :  61
현재 전체 데이터 갯수 :  24979
-------------------------------------
평점 순 링크 순서 :  225
기업 이름 :  한국오라클(유)
기업 리뷰 갯수 :  195
현재 전체 데이터 갯수 :  25174
-------------------------------------
평점 순 링크 순서 :  226
기업 이름 :  한동대학교
기업 리뷰 갯수 :  29
현재 전체 데이터 갯수 :  25203
-------------------------------------
평점 순 링크 순서 :  227
기업 이름 :  (주)리팩
기업 리뷰 갯수 :  49
현재 전체 데이터 갯수 :  25252
-------------------------------------
평점 순 링크 순서 :  228
기업 이름 :  (주)아이티엠코퍼레이션건축사사무소
기업 리뷰 갯수 :  26
현재 전체 데

TimeoutException: Message: 


In [37]:
data

[[0,
  '페이스북코리아(유)',
  '4.7',
  'IT/웹/통신',
  '85%',
  '81%',
  '93%',
  '4.3',
  '4.6',
  '4.0',
  '4.6',
  '4.5',
  '"개발자로서 다양한 경험을 할 수 있고 개발자로서 커리어가 잘 쌓임,\n성과에 따라 대우가 달라짐"',
  '연차를 윗사람 눈치 보지 않고 쓸 수 있음. 도서 구입, 개인 필요 장비 등 지원은 아주 잘 해줌',
  '성과가 너무 명확하고 요즘은 다른 플랫폼에 밀리는 추세라 맨날 눈치를 봐야 함',
  '적극적인 투자가 필요하고, 여러가지로 투자가 많이 필요함과 동시에 새로운 아이디어가 필요함',
  '4',
  'IT/인터넷',
  '전직원',
  '서울',
  '0'],
 [0,
  '페이스북코리아(유)',
  '4.7',
  'IT/웹/통신',
  '85%',
  '81%',
  '93%',
  '4.3',
  '4.6',
  '4.0',
  '4.6',
  '4.5',
  '"스마트한 사람들과 자기주도적으로 일할 수 있는 회사. 존중받으며 일할 수 있는 회사."',
  '자기주도적으로 일할 수 있음 (매니저가 일을 분배해주는 형식이 아니라 자기가 알아서 해야함)\n출퇴근 시간이 자유롭고 성과를 내기만 한다면 업무 터치 없음',
  '성과위주의 문화. 소위 임팩트 없이 일하면 도태되기 싶다.',
  '회사의 사이즈가 계속해서 커지고 있지만 인간적이고 따뜻했던 회사의 문화 계속 가져가주세요',
  '5',
  'IT/인터넷',
  '전직원',
  '서울',
  '0'],
 [0,
  '페이스북코리아(유)',
  '4.7',
  'IT/웹/통신',
  '85%',
  '81%',
  '93%',
  '4.3',
  '4.6',
  '4.0',
  '4.6',
  '4.5',
  '"직원들이 좋은 대우를 받으며 일할수 있는곳같습니다."',
  '우선연봉이 높은편이며 복지가 좋음 사내문화가 수평 문화기때문에 자유롭게 일할수 있는 환경 구성이 되어있습니다',

In [452]:
len(data)

12738

In [20]:
df_result = pd.DataFrame(data, columns=['링크 순서', '기업','평점','산업분류', '직원의 기업 추천율', '직원이 전망하는 성장 가능성','이 기업의 CEO 지지율','승진 기회 및 가능성','복지 및 급여','업무와 삶의 균형','사내문화','경영진','제목','장점','단점','경영진에게 바라는 점' ,'각 리뷰 평점', '직무' ,'재직상태', '지역' ,'리뷰 좋아요 개수'])

In [19]:
# for i in tqdm_notebook(range(len(df_result['제목']))):
#     df_result['제목'][i] = df_result['제목'][i].replace('\n', ' ')
#     df_result['장점'][i] = df_result['장점'][i].replace('\n', ' ')
#     df_result['단점'][i] = df_result['단점'][i].replace('\n', ' ')

In [39]:
df_result

,링크 순서,기업,평점,산업분류,직원의 기업 추천율,직원이 전망하는 성장 가능성,이 기업의 CEO 지지율,승진 기회 및 가능성,복지 및 급여,업무와 삶의 균형,...,경영진,제목,장점,단점,경영진에게 바라는 점,각 리뷰 평점,직무,재직상태,지역,리뷰 좋아요 개수
0,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""개발자로서 다양한 경험을 할 수 있고 개발자로서 커리어가 잘 쌓임,\n성과에 따라...","연차를 윗사람 눈치 보지 않고 쓸 수 있음. 도서 구입, 개인 필요 장비 등 지원은...",성과가 너무 명확하고 요즘은 다른 플랫폼에 밀리는 추세라 맨날 눈치를 봐야 함,"적극적인 투자가 필요하고, 여러가지로 투자가 많이 필요함과 동시에 새로운 아이디어가...",4,IT/인터넷,전직원,서울,0
1,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""스마트한 사람들과 자기주도적으로 일할 수 있는 회사. 존중받으며 일할 수 있는 회사.""",자기주도적으로 일할 수 있음 (매니저가 일을 분배해주는 형식이 아니라 자기가 알아서...,성과위주의 문화. 소위 임팩트 없이 일하면 도태되기 싶다.,회사의 사이즈가 계속해서 커지고 있지만 인간적이고 따뜻했던 회사의 문화 계속 가져가주세요,5,IT/인터넷,전직원,서울,0
2,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""직원들이 좋은 대우를 받으며 일할수 있는곳같습니다.""",우선연봉이 높은편이며 복지가 좋음 사내문화가 수평 문화기때문에 자유롭게 일할수 있는...,정규직과 비정규직 대우 차별이 심함 그리고 아직 사무실 규모가크지않아서 식사가 따로...,없습니다. 어짜피 제가 말해서 바뀌지 않을테니까 말입니다.,5,인사/총무,전직원,서울,1
3,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""합리적인 회사 생활.\n인풋과 아웃풋의 조화.\n국내 대기업과는 완전히 다르다.""","괜찮은 대우, 논리가 통하고\n일에 집중 할 수 있게 한다.\n시설도 좋고 전체적으...",처음에 들어가면 아무도 챙겨주지 않는데\n아마 이것이 외국계 회사의 특징이라 생각이...,조직이 커진다고 국내 대기업 처럼 관리의 조직이 되지 않았으면 합니다.,5,마케팅/시장조사,현직원,해외,0
4,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""누구나 알고있는 유명한 sns회사로써 확실히 다른곳과 다릅니다""","근무환경이 좋으며 분위기가 굉장히 자유롭고, 내 일에 집중할 수 있어서 좋은 곳입니다.",딱히 꼽을 것이 없지만 굳이 한가지를 꼽자면 컴퓨터를 보고 하는 일이기 때문에 본인...,지금처럼 다른 기업들과 다르게 좋은 분위기를 가지며 긍정적인 마인드를 유지해주셨으면...,4,IT/인터넷,전직원,서울,1
5,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""대우좋고 보람있게 일할 수 있는 기업입니다 추천합니다""",분위기좋고 확실한 보상과 수직적이지 않는 분위기가 최고입니다,단점 그다지 별로 없는거같은데 굳이 생각하면 드가기가 너무 힘드네여,앞으로도 더욱 힘내주세요 화이팅입니다.,4,IT/인터넷,전직원,서울,2
6,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""환경은 좋고 직원이 많지 않은데 점점 많아지고 있는추세인것같다.""",복지 복지 음료수 간식거리 공짜 좋은 환경의 오피스 괜찬은 연봉 등등등..........,계약직과 정규직의 차별이 있는 편이다. 대우면에서 나 평소에도...............,딱히 없음......없어요없습니다없어요 없습니다다아아아..,3,서비스/고객지원,전직원,서울,0
7,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""개발자가 대우받고 사람들이 굉장히 좋음 팀워크도 좋음""","윗사람 눈치 안보고 연차 사용, 점심 저녁 제공, 깔끔한 사무실 분위기 등등","낙하산 인사, 인맥... 너무 괴롭다. 일좀 하자... 제발...",일하는 사람 좀 중용했으면 좋겠다. 열심히 하면 할수록 힘빠짐,5,경영/기획/컨설팅,전직원,서울,3
8,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""뭐랄까 굉장히 깨어있는 사람들과 같이 있는 느낌이었습니다. 물들어가는게 좋다랄까요 ?""",발빠른 판단과 협업이 이루어질 수 있는 분위기를 조성합니다. 또한 결과 성취에 너무...,보고를 위한 단계가 간소화되어 자칫 확인을 받아야 할 사항에 대해 쉽게 간과할 수 ...,신규사업에 대한 투자를 직원들과 상의한다면 보다 보람찬 업무 분위기를 조성할 수 있...,4,IT/인터넷,전직원,서울,1
9,0,페이스북코리아(유),4.7,IT/웹/통신,85%,81%,93%,4.3,4.6,4.0,...,4.5,"""대우받으며 보람차게 일할 수 있는 곳 입니다.""","좋은 분위기에서 자기 분야 전문성을 기를 수 있고, 사업 기반이 안정적이고 체계적인...",반복적 업무와 승진이 힘들다는 점이 있지만 큰 단점은 거의 없다고 봐도 무방합니다.,신규 프로젝트 추진과 다양한 승진 프로모션 바랍니다,5,미디어/홍보,전직원,서울,3


In [21]:
df_result.to_csv('잡플래닛_포스코에너지(주)_237.csv')